# Install / Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import os
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import Markdown, display

import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.utils.yaml_utils as yu
import edurel.utils.mermaid as mmu
import edurel.widgets.mermaid_viz as mmw
import edurel.widgets.chatman as cmw
import edurel.utils.misc as mu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"
LOG_DIR = f"{BASE_DIR}/logs"

DB_TAG = "adw-olap"

def run(tag, q, model, schema, db, dbrun=True):
    print(f"{tag}:\n{q}")
    sql = llmu.chat_text_to_sql(model, schema, q)
    print(sql)
    if dbrun:
        db.sql_print(sql)
    return(sql)

# Database

In [6]:
con = dbcu.adw_olap()
additional_fks = {
    "DimAccount": ["DimAccount|ParentAccountKey|AccountKey"],
    "DimDepartmentGroup": ["DimDepartmentGroup|ParentDepartmentGroupKey|DepartmentGroupKey"],
    "DimEmployee": ["DimEmployee|ParentEmployeeKey|EmployeeKey"],
    "DimOrganization": ["DimOrganization|ParentOrganizationKey|OrganizationKey"],
    }

db = dbu.DbHandler(con,additional_fks=additional_fks)
schema_base = db.schema_yaml_str(["nullable", "fkname"])

In [7]:
spec = """
del table pattern: DimAccount
del table pattern: DimDepartmentGroup
del table pattern: DimEmployee
del table pattern: DimOrganization
del table pattern: DimReseller
del table pattern: DimScenario
del table pattern: DimSalesReason
del table pattern: FactCurrencyRate
del table pattern: FactAdditionalInternationalProductDescription
del table pattern: FactCallCenter
del table pattern: FactFinance
del table pattern: FactInternetSalesReason
del table pattern: FactProductInventory
del table pattern: FactResellerSales
del table pattern: FactSalesQuota
del table pattern: FactSurveyResponse
del table pattern: NewFactCurrencyRate
del table pattern: ProspectiveBuyer
del column FactInternetSales index: [8:17, 19:]
del column DimPromotion index: [1, 3, 4, 7, 8, 10, 11]
del column DimProduct index: [1, 3, 4, 6, 7, 8:]
del column DimProductSubcategory index: [1, 3, 4]
del column DimProductCategory index: [1, 3, 4]
del column DimCustomer index: [2, 3, 5, 7:]
del column DimGeography index: [6, 7, 10]

"""
schema_reduced = yu.yaml_to_yaml(schema_base, spec)
schema = f'```yaml\n{schema_reduced}\n```'
print(schema)

```yaml
tables:
- tablename: DimCurrency
  columns:
  - columnname: CurrencyKey
    type: INTEGER
  - columnname: CurrencyAlternateKey
    type: VARCHAR
  - columnname: CurrencyName
    type: VARCHAR
  primary_key:
  - CurrencyKey
- tablename: DimCustomer
  columns:
  - columnname: CustomerKey
    type: INTEGER
  - columnname: GeographyKey
    type: INTEGER
  - columnname: FirstName
    type: VARCHAR
  - columnname: LastName
    type: VARCHAR
  primary_key:
  - CustomerKey
  foreign_keys:
  - sourcecolumns:
    - GeographyKey
    targettable: DimGeography
    targetcolumns:
    - GeographyKey
- tablename: DimDate
  columns:
  - columnname: DateKey
    type: INTEGER
  - columnname: FullDateAlternateKey
    type: DATE
  - columnname: DayNumberOfWeek
    type: INTEGER
  - columnname: EnglishDayNameOfWeek
    type: VARCHAR
  - columnname: SpanishDayNameOfWeek
    type: VARCHAR
  - columnname: FrenchDayNameOfWeek
    type: VARCHAR
  - columnname: DayNumberOfMonth
    type: INTEGER
  - colum

In [8]:
mmu.schema_mermaid_png(schema_reduced, output_path="schema.png", omit_tags=["nullable", "fkname"], direction="LR", scale=6)


## Order date range

In [5]:
sql = """
select min(orderdate) as min_order_date,
       max(orderdate) as max_order_date
from FactInternetSales;
"""
ddbu.sql_print(con, sql)

┌─────────────────────┬─────────────────────┐
│   min_order_date    │   max_order_date    │
│      timestamp      │      timestamp      │
├─────────────────────┼─────────────────────┤
│ 2010-12-29 00:00:00 │ 2014-01-28 00:00:00 │
└─────────────────────┴─────────────────────┘



# Prompts

In [9]:
system_prompt1 = """
You are an expert SQL query generator. 
Convert natural language questions into valid SQL queries.
Use duckdb syntax.
"""

# Questions

## q1

In [10]:
q1 = """
- internet sales amount of all product categories in 2012 on month level and also year over year in 2011
- Output Category, Year, Month, YearPrevious, MonthPrevious, Sales, SalesPrevious
- Sorted by Category, Year, Month
"""

## q2

In [11]:
q2 = """
- internet sales amount of top 3 product subcategories overall and top 3 cities with most customers
- Output Subcategory, City, SalesAmount
- Sorted by Subcategory, City
"""

## q3

In [12]:
q3 = """
- internet sales amount of top 3 product subcategories overall and top 3 cities with most customers
- in the output there might be more than 3 entries because of ties, use ranking function
- Output Subcategory, City, SalesAmount
- Sorted by Subcategory, City
"""

## q4

In [13]:
q4 = """
- internet sales amount of top 3 product subcategories overall and top 3 cities with most customers
- in the output there might be more than 3 entries because of ties, use ranking function
- Output Subcategory, City, SalesAmount
- Sorting should be done as follows
  - Subcategory according to their overall sales amount descending
  - City according to their number of customers descending
"""

## q5

## q6

## q7

## q8

## q9

## q10

## q11

## q12

## q13

## q14

## q15

## q16

## q17

## q18

## q19

## q20

## q21

## q22

## q23

## q24

In [ ]:
q24 = """
create a SQL query that outputs the following columns:
- tag either 'shortest' or 'longest'
- persontype of person
- numberof no of customers in that group
the output should be calculated as follows:
- 'shortest' are customer with minimal modifieddate in the Customer table
- 'longest' are customer with maximal modifieddate in the Customer table    
- regard only customers that have placed orders in 2024
sorted by tag and persontype;
"""

# GEMINI25FLASH

In [17]:
gemini25_flash = llmu.stats_c(llmu.GEMINI25FLASH)


In [18]:
sql = run("q1", q1, gemini25_flash, schema, db, dbrun=True)


q1:

Turn the following question into SQL
- internet sales amount of all product categories in 2012 on month level and also year over year in 2011
- Output Category, Year, Month, YearPrevious, MonthPrevious, Sales, SalesPrevious
- Sorted by Category, Year, Month


WITH MonthlySales AS (
  SELECT
    dpc.EnglishProductCategoryName AS Category,
    dd.CalendarYear AS SalesYear,
    dd.EnglishMonthName AS SalesMonth,
    dd.MonthNumberOfYear AS MonthNum,
    SUM(fis.SalesAmount) AS Sales
  FROM FactInternetSales AS fis
  JOIN DimDate AS dd
    ON fis.OrderDateKey = dd.DateKey
  JOIN DimProduct AS dp
    ON fis.ProductKey = dp.ProductKey
  JOIN DimProductSubcategory AS dps
    ON dp.ProductSubcategoryKey = dps.ProductSubcategoryKey
  JOIN DimProductCategory AS dpc
    ON dps.ProductCategoryKey = dpc.ProductCategoryKey
  WHERE
    dd.CalendarYear IN (2011, 2012)
  GROUP BY
    dpc.EnglishProductCategoryName,
    dd.CalendarYear,
    dd.EnglishMonthName,
    dd.MonthNumberOfYear
)
SELECT
  m

# GEMINI3PRO

In [19]:
gemini3_pro = llmu.stats_c(llmu.GEMINI3PRO)


In [20]:
sql = run("q1", q1, gemini3_pro, schema, db, dbrun=True)

q1:

Turn the following question into SQL
- internet sales amount of all product categories in 2012 on month level and also year over year in 2011
- Output Category, Year, Month, YearPrevious, MonthPrevious, Sales, SalesPrevious
- Sorted by Category, Year, Month


WITH MonthlySales AS (
    SELECT
        pc.EnglishProductCategoryName AS Category,
        d.CalendarYear AS Year,
        d.MonthNumberOfYear AS Month,
        SUM(fis.SalesAmount) AS Sales
    FROM FactInternetSales fis
    JOIN DimDate d ON fis.OrderDateKey = d.DateKey
    JOIN DimProduct p ON fis.ProductKey = p.ProductKey
    JOIN DimProductSubcategory psc ON p.ProductSubcategoryKey = psc.ProductSubcategoryKey
    JOIN DimProductCategory pc ON psc.ProductCategoryKey = pc.ProductCategoryKey
    WHERE d.CalendarYear IN (2011, 2012)
    GROUP BY pc.EnglishProductCategoryName, d.CalendarYear, d.MonthNumberOfYear
)
SELECT
    curr.Category,
    curr.Year,
    curr.Month,
    prev.Year AS YearPrevious,
    prev.Month AS Month

# GLM47

In [ ]:
glm47 = llmu.stats_c(llmu.GLM47)

In [16]:
chat_glm47 = llmc.LLMChat(glm47)
chat_glm47.set_system_prompt(system_prompt1)

## q1

In [19]:
chat_glm47.clear_conversation()
chat_glm47.add_user_message(schema)
content_glm47_q1 = chat_glm47.add_user_message(q1)

In [57]:
sql_glm47_q1 = mu.sql_extract(content_glm47_q1)
output_glm47_q1 = db.sql_str(sql_glm47_q1)
print(output_glm47_q1)

┌─────────────┬───────┬───────┬──────────────┬───────────────┬───────────────┬───────────────┐
│  Category   │ Year  │ Month │ YearPrevious │ MonthPrevious │     Sales     │ SalesPrevious │
│   varchar   │ int32 │ int32 │    int32     │     int32     │ decimal(38,2) │ decimal(38,2) │
├─────────────┼───────┼───────┼──────────────┼───────────────┼───────────────┼───────────────┤
│ Accessories │  2012 │    12 │         2011 │            12 │       2147.08 │          NULL │
│ Bikes       │  2011 │     1 │         2010 │             1 │     469823.94 │          NULL │
│ Bikes       │  2011 │     2 │         2010 │             2 │     466334.93 │          NULL │
│ Bikes       │  2011 │     3 │         2010 │             3 │     485198.69 │          NULL │
│ Bikes       │  2011 │     4 │         2010 │             4 │     502073.88 │          NULL │
│ Bikes       │  2011 │     5 │         2010 │             5 │     561681.51 │          NULL │
│ Bikes       │  2011 │     6 │         2010 │    

In [58]:
type(output_glm47_q1)

str

In [74]:
chat_glm47.insert_message(1, f"```\n{output_glm47_q1}```")

True

In [81]:
chat_glm47.insert_message_at_end("xxx")

In [73]:
chat_glm47.delete_message(1)  


True

In [ ]:
chat_glm47.get_conversation_length()

1

In [ ]:
file_path = chat_glm47.log_conversation(LOG_DIR, DB_TAG, llmu.GLM47, "q1", "v1")
print(f"Conversation logged to: {file_path}")

In [83]:
display(Markdown(chat_glm47.show_conversation(lastn_only=2)))

[3] USER:
 bbb

[4] USER:
 xxx

## q2

In [67]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q2)

In [69]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌────────────────┬─────────┬───────────────┐
│  Subcategory   │  City   │  SalesAmount  │
│    varchar     │ varchar │ decimal(38,2) │
├────────────────┼─────────┼───────────────┤
│ Mountain Bikes │ Concord │    9952760.77 │
│ Mountain Bikes │ London  │    9952760.77 │
│ Mountain Bikes │ Paris   │    9952760.77 │
│ Road Bikes     │ Concord │   14520583.50 │
│ Road Bikes     │ London  │   14520583.50 │
│ Road Bikes     │ Paris   │   14520583.50 │
│ Touring Bikes  │ Concord │    3844801.05 │
│ Touring Bikes  │ London  │    3844801.05 │
│ Touring Bikes  │ Paris   │    3844801.05 │
└────────────────┴─────────┴───────────────┘



## q3

In [ ]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q3)

In [ ]:
sql = mu.sql_extract(r)
db.sql_print(sql)

## q4

In [70]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q4)

In [72]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌────────────────┬─────────┬───────────────┐
│  Subcategory   │  City   │  SalesAmount  │
│    varchar     │ varchar │ decimal(38,2) │
├────────────────┼─────────┼───────────────┤
│ Road Bikes     │ London  │   14520583.50 │
│ Road Bikes     │ Paris   │   14520583.50 │
│ Road Bikes     │ Concord │   14520583.50 │
│ Road Bikes     │ Burien  │   14520583.50 │
│ Mountain Bikes │ London  │    9952760.77 │
│ Mountain Bikes │ Paris   │    9952760.77 │
│ Mountain Bikes │ Concord │    9952760.77 │
│ Mountain Bikes │ Burien  │    9952760.77 │
│ Touring Bikes  │ London  │    3844801.05 │
│ Touring Bikes  │ Paris   │    3844801.05 │
│ Touring Bikes  │ Concord │    3844801.05 │
│ Touring Bikes  │ Burien  │    3844801.05 │
├────────────────┴─────────┴───────────────┤
│ 12 rows                        3 columns │
└──────────────────────────────────────────┘



# OPUS45

In [7]:
opus45 = llmu.stats_c(llmu.OPUS45)

In [ ]:
chat = llmc.LLMChat(opus45)
chat.set_system_prompt(system_prompt1)

## q1

In [ ]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q1)

In [18]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌─────────────┬───────┬───────┬──────────────┬───────────────┬───────────────┬───────────────┐
│  Category   │ Year  │ Month │ YearPrevious │ MonthPrevious │     Sales     │ SalesPrevious │
│   varchar   │ int64 │ int64 │    int64     │     int64     │ decimal(38,2) │ decimal(38,2) │
├─────────────┼───────┼───────┼──────────────┼───────────────┼───────────────┼───────────────┤
│ Accessories │  2012 │    12 │         NULL │          NULL │       2147.08 │          NULL │
│ Bikes       │  2012 │     1 │         2011 │             1 │     495364.01 │     469823.94 │
│ Bikes       │  2012 │     2 │         2011 │             2 │     506994.08 │     466334.93 │
│ Bikes       │  2012 │     3 │         2011 │             3 │     373482.95 │     485198.69 │
│ Bikes       │  2012 │     4 │         2011 │             4 │     400335.59 │     502073.88 │
│ Bikes       │  2012 │     5 │         2011 │             5 │     358877.87 │     561681.51 │
│ Bikes       │  2012 │     6 │         2011 │    

## q2

In [32]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q2)

In [37]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌────────────────┬────────────┬───────────────┐
│  Subcategory   │    City    │  SalesAmount  │
│    varchar     │  varchar   │ decimal(38,2) │
├────────────────┼────────────┼───────────────┤
│ Mountain Bikes │ Bellingham │      81316.70 │
│ Mountain Bikes │ Burien     │      65082.22 │
│ Mountain Bikes │ Concord    │      61956.51 │
│ Road Bikes     │ Bellingham │      82252.22 │
│ Road Bikes     │ Burien     │     115466.70 │
│ Road Bikes     │ Concord    │     104500.44 │
│ Touring Bikes  │ Bellingham │      33626.70 │
│ Touring Bikes  │ Burien     │      38303.58 │
│ Touring Bikes  │ Concord    │      33738.09 │
└────────────────┴────────────┴───────────────┘



In [ ]:
chat.add_user_message(schema)
r = chat.add_user_message(q2)

## q3

In [52]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q3)

In [54]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌────────────────┬─────────┬───────────────┐
│  Subcategory   │  City   │  SalesAmount  │
│    varchar     │ varchar │ decimal(38,2) │
├────────────────┼─────────┼───────────────┤
│ Mountain Bikes │ Burien  │      65082.22 │
│ Mountain Bikes │ Concord │      61956.51 │
│ Mountain Bikes │ London  │     300619.75 │
│ Mountain Bikes │ Paris   │     186654.04 │
│ Road Bikes     │ Burien  │     115466.70 │
│ Road Bikes     │ Concord │     104500.44 │
│ Road Bikes     │ London  │     341432.71 │
│ Road Bikes     │ Paris   │     276410.82 │
│ Touring Bikes  │ Burien  │      38303.58 │
│ Touring Bikes  │ Concord │      33738.09 │
│ Touring Bikes  │ London  │     135471.93 │
│ Touring Bikes  │ Paris   │      57671.49 │
├────────────────┴─────────┴───────────────┤
│ 12 rows                        3 columns │
└──────────────────────────────────────────┘



## q4

In [59]:
chat.clear_conversation()
chat.add_user_message(schema)
r = chat.add_user_message(q4)

In [61]:
sql = mu.sql_extract(r)
db.sql_print(sql)

┌────────────────┬─────────┬───────────────┐
│  Subcategory   │  City   │  SalesAmount  │
│    varchar     │ varchar │ decimal(38,2) │
├────────────────┼─────────┼───────────────┤
│ Road Bikes     │ London  │     341432.71 │
│ Road Bikes     │ Paris   │     276410.82 │
│ Road Bikes     │ Burien  │     115466.70 │
│ Road Bikes     │ Concord │     104500.44 │
│ Mountain Bikes │ London  │     300619.75 │
│ Mountain Bikes │ Paris   │     186654.04 │
│ Mountain Bikes │ Burien  │      65082.22 │
│ Mountain Bikes │ Concord │      61956.51 │
│ Touring Bikes  │ London  │     135471.93 │
│ Touring Bikes  │ Paris   │      57671.49 │
│ Touring Bikes  │ Burien  │      38303.58 │
│ Touring Bikes  │ Concord │      33738.09 │
├────────────────┴─────────┴───────────────┤
│ 12 rows                        3 columns │
└──────────────────────────────────────────┘



# Chat History

In [71]:
display(Markdown(chat.show_conversation(lastn_only=2)))

[3] USER:
 
- internet sales amount of top 3 product subcategories overall and top 3 cities with most customers
- in the output there might be more than 3 entries because of ties, use ranking function
- Output Subcategory, City, SalesAmount
- Sorting should be done as follows
  - Subcategory according to their overall sales amount descending
  - City according to their number of customers descending


[4] AI:
 ```sql
WITH TopSubcategories AS (
    SELECT 
        dpsc.EnglishProductSubcategoryName AS Subcategory,
        SUM(fis.SalesAmount) AS TotalSalesAmount,
        RANK() OVER (ORDER BY SUM(fis.SalesAmount) DESC) AS SubcategoryRank
    FROM 
        FactInternetSales fis
        JOIN DimProduct dp ON fis.ProductKey = dp.ProductKey
        JOIN DimProductSubcategory dpsc ON dp.ProductSubcategoryKey = dpsc.ProductSubcategoryKey
    GROUP BY 
        dpsc.EnglishProductSubcategoryName
),
TopCities AS (
    SELECT 
        dg.City,
        COUNT(DISTINCT dc.CustomerKey) AS CustomerCount,
        RANK() OVER (ORDER BY COUNT(DISTINCT dc.CustomerKey) DESC) AS CityRank
    FROM 
        DimCustomer dc
        JOIN DimGeography dg ON dc.GeographyKey = dg.GeographyKey
    GROUP BY 
        dg.City
)
SELECT 
    tsc.Subcategory,
    tc.City,
    tsc.TotalSalesAmount AS SalesAmount
FROM 
    TopSubcategories tsc
CROSS JOIN 
    TopCities tc
WHERE 
    tsc.SubcategoryRank <= 3
    AND tc.CityRank <= 3
ORDER BY 
    tsc.TotalSalesAmount DESC,
    tc.CustomerCount DESC;
```

# SQL Run

In [ ]:
db.sql_print("""

""")